In [4]:
from google import genai
from PIL import Image
import os
import re
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
image = Image.open('image.png')

In [6]:
client = genai.Client(api_key=os.getenv("GENAI_API_KEY"))

In [7]:
prompt = (
    "Transcribe the nutrition facts table in this image and output the data "
    "as a single **JSON object**. Use keys like 'serving-size', 'energy-kcal', 'fat', 'carbohydrates', 'proteins', 'saturated-fat', 'trans-fat', 'sugars', 'added-sugars', 'sodium', 'salt', and 'fiber'. "
    "Do not include any text outside of the JSON object."
    "If the values are not in the image, fill the values with 0"
    "Pay attention to the unit, normalize the unit so it's stated in g (gram) instead of mg (miligram)"
)

In [8]:
result = client.models.generate_content(
    model='gemini-2.5-flash', 
    contents=[prompt, image]
)

In [9]:
data = result.text
print(data)

```json
{
  "serving-size": "21g",
  "energy-kcal": 60,
  "fat": 0,
  "carbohydrates": 17,
  "proteins": 0,
  "saturated-fat": 0,
  "trans-fat": 0,
  "sugars": 17,
  "added-sugars": 17,
  "sodium": 0,
  "salt": 0,
  "fiber": 0
}
```


In [10]:
cleanedData = data.strip().replace('```json', '').replace('```', '').replace('\n','').strip()
print(cleanedData)

cleanedData = re.sub(r'\s+', ' ', cleanedData).strip()

{  "serving-size": "21g",  "energy-kcal": 60,  "fat": 0,  "carbohydrates": 17,  "proteins": 0,  "saturated-fat": 0,  "trans-fat": 0,  "sugars": 17,  "added-sugars": 17,  "sodium": 0,  "salt": 0,  "fiber": 0}


In [11]:
tempDict = {}

for i in cleanedData.split(','):
    x = i.split(':')
    col = (x[0].split('"'))[1]
    num = x[1]

    tempDict[col] = num

    print(i)

{ "serving-size": "21g"
 "energy-kcal": 60
 "fat": 0
 "carbohydrates": 17
 "proteins": 0
 "saturated-fat": 0
 "trans-fat": 0
 "sugars": 17
 "added-sugars": 17
 "sodium": 0
 "salt": 0
 "fiber": 0}


In [12]:
resDict = {}

for i in tempDict:
    if i == 'serving-size':
        divider = tempDict[i].split('"')
        divider = divider[1].split('g')
        divider = int(divider[0].strip())
    else:
        try:
            resDict[i+'_1g'] =  round(float(tempDict[i].strip())/divider, 3)
        except ValueError:
            value = tempDict[i].split("}")
            value = float(value[0].strip())

            resDict[i+'_1g'] =  round(value/divider, 3)

resDict

{'energy-kcal_1g': 2.857,
 'fat_1g': 0.0,
 'carbohydrates_1g': 0.81,
 'proteins_1g': 0.0,
 'saturated-fat_1g': 0.0,
 'trans-fat_1g': 0.0,
 'sugars_1g': 0.81,
 'added-sugars_1g': 0.81,
 'sodium_1g': 0.0,
 'salt_1g': 0.0,
 'fiber_1g': 0.0}